In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [3]:
from flask import Flask, request, jsonify
import base64
from PIL import Image
import io
import numpy as np
import tensorflow as tf
import requests
import cv2
import pytesseract
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM
from tensorflow.keras.applications import EfficientNetB0

ImportError: libcupti.so.12: cannot open shared object file: No such file or directory

In [ ]:
app = Flask(__name__)

# Initialize T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Initialize Sentence Transformer for retrieval
retriever = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize EfficientNet for image processing
efficient_net = EfficientNetB0(include_top=False, weights='imagenet')

@app.route('/process-image', methods=['POST'])
def process_image():
    data = request.json
    image_data = data.get('imageData')
    action = data.get('action')  # "add" or "update"
    task_id = data.get('taskId')  # Only used for updating
    
    # Decode base64 string to image
    image = decode_base64_image(image_data)
    
    # Extract text using OCR
    extracted_text = extract_text(image)
    
    # Retrieve related documents and predict task details
    task_details = predict_task_details(extracted_text)
    
    # Determine the action and call the appropriate API
    if action == "add":
        response = add_task(task_details)
    elif action == "update" and task_id:
        response = update_task(task_id, task_details)
    else:
        response = {"status": "error", "message": "Invalid action or missing taskId"}
    
    return jsonify(response)


In [ ]:



def decode_base64_image(base64_str):
    decoded_data = base64.b64decode(base64_str)
    np_data = np.frombuffer(decoded_data, np.uint8)
    image = cv2.imdecode(np_data, cv2.IMREAD_COLOR)
    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

def extract_text(image):
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    extracted_text = pytesseract.image_to_string(gray_image)
    return extracted_text

def predict_task_details(extracted_text):
    related_docs = retrieve_related_documents(extracted_text)
    
    combined_input = f"Task details: {extracted_text} {related_docs}"
    
    inputs = t5_tokenizer.encode(combined_input, return_tensors="pt", max_length=512, truncation=True)
    outputs = t5_model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    generated_text = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    task_details = parse_generated_text(generated_text)
    
    # Ensure priority and difficulty are between 1 and 5
    task_details['priority'] = max(1, min(5, task_details.get('priority', 1)))
    task_details['difficulty'] = max(1, min(5, task_details.get('difficulty', 1)))
    
    return task_details

def retrieve_related_documents(query):
    # Example function to retrieve related documents or knowledge
    related_docs = "Example related document content"
    return related_docs

def parse_generated_text(generated_text):
    # Implement parsing logic based on your needs
    task_details = {
        "title": "Predicted Title",
        "note": "Generated Note",
        "type": "Generated Type",
        "date": "Predicted Date",
        "beginTime": "Predicted BeginTime",
        "endTime": "Predicted EndTime",
        "priority": extract_priority_from_text(generated_text),  # Extract priority from text
        "difficulty": extract_difficulty_from_text(generated_text),  # Extract difficulty from text
    }
    return task_details

def extract_priority_from_text(text):
    # Example logic to extract priority from text
    # Replace with actual extraction logic
    return int(text.split('priority:')[1].strip().split()[0])

def extract_difficulty_from_text(text):
    # Example logic to extract difficulty from text
    # Replace with actual extraction logic
    return int(text.split('difficulty:')[1].strip().split()[0])




In [ ]:
def add_task(task_details):
    url = "http://127.0.0.1:3000/api/tasks/tasks"
    response = requests.post(url, json=task_details)
    if response.status_code == 201:
        return {"status": "success", "message": "Task added successfully"}
    else:
        return {"status": "error", "message": f"Failed to add task: {response.status_code}"}

def update_task(task_id, task_details):
    url = f"http://127.0.0.1:3000/api/tasks/tasks/{task_id}"
    response = requests.put(url, json=task_details)
    if response.status_code == 200:
        return {"status": "success", "message": "Task updated successfully"}
    else:
        return {"status": "error", "message": f"Failed to update task: {response.status_code}"}


In [ ]:
def create_advanced_model():
    base_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    base_model.trainable = False
    
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Add LSTM layer
    x = LSTM(256, return_sequences=False)(x)
    
    # Output layers for task details
    priority_output = Dense(1, activation='sigmoid')(x)  # Adjust based on your range (1-5)
    difficulty_output = Dense(1, activation='sigmoid')(x)  # Adjust based on your range (1-5)
    
    model = Model(inputs=base_model.input, outputs=[priority_output, difficulty_output])
    model.compile(optimizer='adam', loss=['mse', 'mse'], metrics=['accuracy'])
    
    return model

if __name__ == '__main__':
    app.run(debug=True)
